In [1]:
!pip install -U firebase-admin langchain-community langchain-google-genai langchain-chroma langchain langgraph tavily-python langchain-huggingface langchain-tavily chromadb gradio

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.

   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   ---- ----------------------------------- 0.3/2.5 MB ? eta -:--:--
   ---- ----------------------------------- 0.3/2.5 MB ? eta -:--:--
   -------- ------------------------------- 0.5/2.5 MB 488.7 kB/s eta 0:00:05
   -------- ------------------------------- 0.5/2.5 MB 488.7 kB/s eta 0:00:05
   ------------ --------------------------- 0.8/2.5 MB 515.8 kB/s eta 0:00:04
   ---------------- ----------------------- 1.0/2.5 MB 562.6 kB/s eta 0:00:03
   ---------------- ----------------------- 1.0/2.5 MB 562.6 kB/s eta 0:00:03
   -------------------- ------------------- 1.3/2.5 MB 566.2 kB/s eta 0:00:03
   -------------------- ------------------- 1.3/2.5 MB 566.2 kB/s eta 0:00:03
   ------------------------ ------------


langchain-openai 0.3.35 requires langchain-core<1.0.0,>=0.3.78, but you have langchain-core 1.2.1 which is incompatible.

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from pydantic import BaseModel, Field

In [ ]:
import os
import getpass

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter API key:")

In [ ]:
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0,
    max_tokens=None,
)

In [ ]:
from langchain_core.documents import Document

In [ ]:
documents = []
for filename in os.listdir():
  if filename.endswith('.txt'):
    with open(filename, 'r', encoding='utf-8') as file:
      content = file.read()
      documents.append(
          Document(
              page_content=content,
              metadata= {
                  'source': filename
              }
          )
      )

In [ ]:
documents[0]

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1200,
    chunk_overlap=200
)

In [ ]:
all_splits = text_splitter.split_documents(documents)

In [ ]:
all_splits[0]

In [ ]:
#vector store
from langchain_chroma import Chroma
#vector embedding model
from langchain_huggingface import HuggingFaceEmbeddings

In [ ]:
!pip install sentence-transformers

In [ ]:
model_name = "sentence-transformers/static-similarity-mrl-multilingual-v1"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}

hf_embedding = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

In [ ]:
vectorstore = Chroma.from_documents(
    documents=all_splits,
    embedding=hf_embedding,
    collection_name="chillstay_docs"
)

In [ ]:
retriver = vectorstore.similarity_search(
    "đăng ký tài khoản", k = 4
)
retriver

In [ ]:
from pydantic import BaseModel, Field
from typing import Type, Optional
from langchain.tools import BaseTool
from langchain.callbacks.manager import (
    CallbackManagerForToolRun,
)

In [ ]:
class RetriveInput(BaseModel):
  query: str = Field("", description="Search query string entered by the user")

In [ ]:
class RetrieveTool(BaseTool):
  name: str = 'chillstay_knowledge_base'
  description: str = """Công cụ này chứa toàn bộ kiến thức về ứng dụng Chillstay.
    Sử dụng công cụ này khi người dùng hỏi về:
    - Hướng dẫn đăng ký, đăng nhập
    - Cách đặt phòng, tìm kiếm
    - Chính sách thanh toán, hủy phòng
    - Các tính năng của app
    - Xử lý lỗi
    
    LUÔN sử dụng công cụ này ĐẦU TIÊN khi hỏi về Chillstay."""
    
  args_schema: Type[BaseModel] = RetriveInput
  return_direct: bool = False

  _vector_store: Type[Chroma]
  _k: int

  def __init__(self, vectorstore, **kwargs) -> None:
    super().__init__(**kwargs)
    self._vector_store = vectorstore
    self._k = kwargs.get("k", 4)

  def _run(
      self,
      query: str,
      run_manager: Optional[CallbackManagerForToolRun] = None,
  ) -> str:
    results = self._vector_store.similarity_search(query, k=self._k)
    documents = [doc.page_content for doc in results]

    return "\n\n".join(documents)



In [ ]:
chillstay_tool = RetrieveTool(vectorstore=vectorstore, k=4)

print(tool.invoke({"query": "hướng dẫn dùng app"}))

# Search web tool

In [ ]:
if "TAVILY_API_KEY" not in os.environ:
    os.environ["TAVILY_API_KEY"] = getpass.getpass("Enter API key:")

In [ ]:
from langchain_tavily import TavilySearch

In [ ]:
search_web_tool = TavilySearch(
    max_results=2,
    topic = "general", # news, finance
)

In [ ]:
search_web_tool.invoke({"query" : "Gía xăng Việt Nam này 17/10/2025"})

In [ ]:
from langgraph.prebuilt import create_react_agent

In [ ]:
agent = create_react_agent(llm, [chillstay_tool, search_web_tool])

ser_input = "Giới thiệu về Einstein"

# stream
for step in agent.stream(
    {"messages": [{"role": "user", "content": user_input}]},
    stream_mode="values"
):
    # tuỳ agent/step structure, step['messages'] có thể là list of Message objects
    step['messages'][-1].pretty_print()

In [ ]:
result = agent.invoke({"messages": [{"role": "user", "content": user_input}]})
print(result['messages'][-1].content)

# Connect firebase

In [ ]:
def initialize_firebase(service_account_path):
    """
    Khởi tạo Firebase Admin SDK
    service_account_path: đường dẫn đến file JSON credentials từ Firebase Console
    """
    try:
        if not firebase_admin._apps:
            cred = credentials.Certificate(service_account_path)
            firebase_admin.initialize_app(cred)
        db = firestore.client()
        print("✓ Firebase đã được khởi tạo thành công!")
        return db
    except Exception as e:
        print(f"✗ Lỗi khởi tạo Firebase: {e}")
        return None

# UI Chatbot

In [ ]:
def chat(user_message, history):
    if history is None:
        history = []
    
    messages = history.copy()
    messages.append({"role": "user", "content": user_message})
    
    try:
        result = agent.invoke({"messages": messages})
        bot_reply = result['messages'][-1].content
        
        history.append({"role": "user", "content": user_message})
        history.append({"role": "assistant", "content": bot_reply})
        
        return "", history
    except Exception as e:
        error_msg = f"Lỗi: {str(e)}"
        history.append({"role": "user", "content": user_message})
        history.append({"role": "assistant", "content": error_msg})
        return "", history

In [ ]:
import gradio as gr

In [ ]:
with gr.Blocks() as demo:
  gr.Markdown("Chatbot private")
  chatbot = gr.Chatbot(type="messages")
  with gr.Row():
    txt = gr.Textbox(
        show_label=False,
        placeholder="Nhập tin nhắn..."
    )
  txt.submit(chat, [txt, chatbot], [txt, chatbot])

demo.launch(share=True)